# MIR Project Phase 2 -- Knn

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import WordNetLemmatizer
from collections import Counter
import csv
import numpy as np
import math

<div dir="rtl">ابتدا متن را آماده میکنیم که این کار مشابه فاز اول انجام شده است با این تفاوت که تمام flag ها را true گذاشته ایم.</div>

In [2]:
def prepare_text(address, delete_punctuation, delete_stopWords, stemming):
        documents = []
        totalDocs = ""
        with open(address, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                doc = {}
                doc['title'] = row['title']
                doc['description'] = row['description']
                doc['views'] = row['views']
                totalDocs += row['description'].lower()+" "+row['title'].lower()
                documents.append(doc)

        totalDocs = nltk.word_tokenize(totalDocs)
        totalDocs = [i for i in totalDocs
                        if i.isalpha()]
        word = Counter(totalDocs)
        stopwords = [i[0] for i in word.most_common(37)]
        #print(stopwords)
        
        tokens = {}
        for i in range(len(documents)):
            doc = documents[i]
            title = doc.get('title').lower()
            title = nltk.word_tokenize(title)
            desc = doc.get('description').lower()
            desc = nltk.word_tokenize(desc)

            #### deleting punctuations
            if delete_punctuation=="T":
                title = [i
                for i in title
                    if i.isalpha()]
                desc = [i
                for i in desc
                    if i.isalpha()]

            ### finding and deleting stopwords
            if delete_stopWords=="T":
                title = [i
                for i in title
                    if i not in stopwords]
                if 'the' in title:
                    print('1')
                desc = [i
                for i in desc
                    if i not in stopwords]
            
            #### stemming
            if stemming=="T":
                stemmer = SnowballStemmer("english")
                title = [stemmer.stem(i)
                for i in title]
                desc = [stemmer.stem(i)
                for i in desc]
            
            #calculating the df of each word
            temp = set()
            for word in title:
                    temp.add(word)
            for word in desc:
                    temp.add(word)
            for word in temp:
                if word in tokens.keys():
                    tokens[word][0] += 1
                else:
                    tokens[word] = [1, len(tokens)]
            #changing the document to processed one
            documents[i]['title'] = title
            documents[i]['description'] = desc
       
        
        return documents, tokens

<div dir="rtl">در این جا ما داده‌های مربوط به train را از فایل مربوطه می‌خوانیم و مراحل پیش‌پردازش را روی آن انجام می‌دهیم و در نهایت برای اطمینان، آن را چاپ می‌کنیم.</div>

In [3]:
documents, tokens = prepare_text("data/train.csv", "T", "T", "T")
print(len(tokens))
print(documents)

10215
[{'title': ['power', 'poem', 'parkinson', 'grow', 'older'], 'description': ['when', 'poet', 'robin', 'morgan', 'found', 'herself', 'face', 'parkinson', 's', 'diseas', 'distil', 'experi', 'into', 'these', 'four', 'quiet', 'power', 'poem', 'medit', 'age', 'loss', 'simpl', 'power', 'notic'], 'views': '-1'}, {'title': ['glamour'], 'description': ['time', 'cultur', 'critic', 'virginia', 'postrel', 'muse', 'true', 'mean', 'power', 'use', 'glamour', 'which', 'defin', 'ani', 'calcul', 'care', 'polish', 'imag', 'design', 'impress', 'persuad'], 'views': '-1'}, {'title': ['everyth', 'hear', 'film', 'lie'], 'description': ['sound', 'design', 'built', 'decept', 'when', 'watch', 'movi', 'or', 'tv', 'show', 'near', 'all', 'sound', 'hear', 'fake', 'taso', 'frantzola', 'explor', 'role', 'sound', 'storytel', 'demonstr', 'just', 'easili', 'brain', 'fool', 'hear'], 'views': '1'}, {'title': ['let', 'parent', 'taboo'], 'description': ['publish', 'rufus', 'griscom', 'alisa', 'volkman', 'live', 'expos',

<div dir="rtl">در این بخش، تابعی برای محاسبه‌ی idf آورده شده که دقیقا مشابه فاز قبلی است. سپس مثالی برای سنجش صحت عملکرد آن آورده شده است.</div>

In [4]:
def idf_calculator(tokens, term, N):
    if(tokens.get(term) != None):
        df = tokens.get(term)[0]
        idf = math.log(N/df)
    else:
        return 0
    return idf

In [5]:
idf_calculator(tokens, 'robin', len(documents))

5.65904658081481

<div dir="rtl">در این قسمت، تابعی طراحی شده که به کمک تابع قبلی، مقدار tf-idf را برای یک داکیومنت انجام می‌دهد.
<br>
در ادامه نیز یک تست برای این تابع انجام شده است.
</div>

In [6]:
def tfidfwighter_knn(tokens, doc, N):
    text = []
    vector = np.zeros((len(tokens),))
    text = doc.get('title') + doc.get('description')
    for word in text:
        tf = text.count(word)
        idf = idf_calculator(tokens, word , N)
        if(tokens.get(word) != None):
            vector[tokens.get(word)[1]] = tf*idf
    return vector

In [7]:
tfidfwighter_knn(tokens,documents[0], len(documents))

array([3.3817793 , 5.94672865, 5.17353877, ..., 0.        , 0.        ,
       0.        ])

<div dir="rtl">
    تابع زیر، فاصله‌ی برداری دو بردار را محاسبه می‌کند.
</div>

In [8]:
def find_distance(doc1, doc2, training_tokens, N):
    a1 = tfidfwighter_knn(training_tokens,doc1, N)
    a2 = tfidfwighter_knn(training_tokens,doc2, N)
    ans = np.linalg.norm(a1-a2)
    return ans

In [9]:
find_distance(documents[0],documents[6], tokens, len(documents))

37.565681734997895

<div dir="rtl">
تابع زیر، مقدار knn را محاسبه می‌کند. ورودی‌های این تابع به ترتیب عدد k، سندی که می‌خواهیم آن را لیبل بزنیم،  tokenها هستند.
    <br>
    این تابع، یک دیکشنری به طولی برابر با k نگهداری می‌کند که همواره شامل داده‌هایی به شکل زیر است که نشان‌دهنده‌ی فاصله با نزدیک‌ترین همسایگان (تا جایی که بررسی شده) و همین‌طور لیبل هر کدام از آن همسایه‌هاست.
    <br>
</div>
ans = {number(from 0 to k-1) : [distance, label]}
<div dir="rtl">
<br>
    این دیکشنری مرتبا به روز می‌شود و در انتهای کار، با توجه به اکثریت مقدار لیبل‌ها در آن، در مورد لیبل برای سند فعلی تصمیم‌گیری می‌شود.
</div>

In [10]:
def knn(k, new_doc, docs, tokens): # k is the number and doc is the array of terms after pre-proccess
    ans = {}    # always saves k nearest neighbours. {number: [distance, label]}
    i = 0
    maximum_indx = 0
    for d in docs:
        current_distance = find_distance(new_doc, d, tokens, len(documents))
        
        if len(ans) < k:
            ans[i] = [current_distance, d['views']]            
            if current_distance > ans[maximum_indx][0]:
                maximum_indx = i
            i += 1
        elif current_distance < ans[maximum_indx][0]:
            ans[maximum_indx] = [current_distance, d['views']]
            maximum_indx = 0
            for l in range(k):
                if ans[l][0] > ans[maximum_indx][0]:
                    maximum_indx = l
    
    s = 0
    for a in ans:
        s += int(ans[a][1])
    if s < 0:
        return -1
    if s > 0:
        return 1
    return 1

In [11]:
knn(1, documents[0], documents, tokens)

-1

<div dir="rtl">
در این بخش، این الگوریتم روی داده‌های train اجرا شده و مقدار accuracy آن در نهایت پرینت می‌شود.
</div>

In [12]:
true = 0
label = [0 for i in range(len(documents))]
for i in range(len(documents)):
    label[i] = knn(5, documents[i], documents, tokens)
    if documents[i].get('views') == str(label[i]):
        true += 1
print(true/len(documents))

0.7599128540305011


<div dir="rtl">
در این بخش، این الگوریتم روی داده‌های test اجرا شده و مقدار accuracy آن در نهایت پرینت می‌شود.
</div>

In [13]:
test_documents, test_tokens = prepare_text("data/test.csv", "T", "T", "T")
test_true = 0
test_label = [0 for i in range(len(test_documents))]
for i in range(len(test_documents)):
    test_label[i] = knn(5, test_documents[i], documents, tokens)
    if test_documents[i].get('views') == str(test_label[i]):
        test_true += 1
print(test_true/len(test_documents))

0.615686274509804


<div dir="rtl">
    در این‌جا دو تابع داریم که به ترتیب معیارهای مربوط به بخش سوم را برای کلاس‌های ۱ و ۱- محاسبه می‌کنند. نتایج این محاسبات، در انتها آمده است.
</div>

In [14]:
def evaluater_for_class_one(test_docs, predicted_labels, name_of_classifier):
    tn, tp,fn,fp = 0,0,0,0
    for i in range(len(predicted_labels)):
        #positive
        if test_docs[i].get('views') == '1':
            #true positive
            if test_docs[i].get('views') == str(predicted_labels[i]):
                tp += 1
            else:
                fp += 1
        #negative
        else:
            #true negative
            if test_docs[i].get('views') == str(predicted_labels[i]):
                tn += 1
            else:
                fn += 1         
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    accuracy = (tp + tn) / (tp+tn+fn+fp)
    f1_score = 2 * precision * recall / (precision + recall)
    print("the evaluation of " + name_of_classifier + " related to the class 1:" )
    print("recall = " + str(recall) )
    print("precision = "+ str(precision) )
    print("accuracy = " + str(accuracy) )
    print("f1_score = " + str(f1_score) )

In [15]:
def evaluater_for_class_minus_one(test_docs, predicted_labels, name_of_classifier):
    tn, tp,fn,fp = 0,0,0,0
    for i in range(len(predicted_labels)):
        #positive
        if test_docs[i].get('views') == '-1':
            #true positive
            if test_docs[i].get('views') == str(predicted_labels[i]):
                tp += 1
            else:
                fp += 1
        #negative
        else:
            #true negative
            if test_docs[i].get('views') == str(predicted_labels[i]):
                tn += 1
            else:
                fn += 1         
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    accuracy = (tp + tn) / (tp+tn+fn+fp)
    f1_score = 2 * precision * recall / (precision + recall)
    print("the evaluation of " + name_of_classifier + " related to the class -1:" )
    print("recall = " + str(recall) )
    print("precision = "+ str(precision) )
    print("accuracy = " + str(accuracy) )
    print("f1_score = " + str(f1_score) )

In [16]:
evaluater_for_class_one(test_documents, test_label, "Knn")

the evaluation of Knn related to the class 1:
recall = 0.5950413223140496
precision = 0.5950413223140496
accuracy = 0.615686274509804
f1_score = 0.5950413223140496


In [17]:
evaluater_for_class_minus_one(test_documents, test_label, "Knn")

the evaluation of Knn related to the class -1:
recall = 0.6343283582089553
precision = 0.6343283582089553
accuracy = 0.615686274509804
f1_score = 0.6343283582089553
